In [1]:
import pandas as pd
import numpy as np

from pythainlp import word_tokenize
import re

import tensorflow as tf

from sklearn import preprocessing
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.metrics import classification_report

from sklearn.feature_extraction.text import TfidfVectorizer
import keras
from keras import *
from keras.layers import *

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm

from imblearn.over_sampling import SMOTE

from sklearn.utils.class_weight import compute_class_weight

import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_excel('course_review_dataset.xlsx')

In [3]:
df = df.dropna()

In [4]:
df.isna().sum()

text           0
score          0
course_name    0
dtype: int64

In [5]:
df

,text,score,course_name
0,โค้ชเอ๋แน่นมากทั้ง MM Mind Set และ Method เทีย...,5,ทำกำไรในตลาดทองอย่างยั่งยืนด้วยเทคนิค 3M
1,เยี่ยมเลยครับ คุณเอ๋ทำ tfex ด้วยนะครับ รอนะครับ,5,ทำกำไรในตลาดทองอย่างยั่งยืนด้วยเทคนิค 3M
2,พี่เอ๋ ใช้เทคนิคง่ายไม่ซับซ้อน ดีครับ,5,ทำกำไรในตลาดทองอย่างยั่งยืนด้วยเทคนิค 3M
3,ได้แนวคิดดีมากเลยค่ะ,5,ทำกำไรในตลาดทองอย่างยั่งยืนด้วยเทคนิค 3M
4,โค้ชเอ๋ ถ่ายทอดเป็นระบบ ตามขั้นตอน ชัดเจน เข้า...,5,ทำกำไรในตลาดทองอย่างยั่งยืนด้วยเทคนิค 3M
...,...,...,...
994,เป็นคอร์สที่ไม่เหมาะสำหรับผู้ไม่มีพื้นฐาน เนื่...,2,Basic Network และ Lab Config Cisco
995,เป็นหลักสูตรที่ดีครับ สงสงตรงไหนสอบถามอาจารย์ไ...,5,Basic Network และ Lab Config Cisco
996,เหมาะกับคนที่พื้นฐาน,2,Basic Network และ Lab Config Cisco
997,ลำบากมือใหม่อย่างผมมากครับ,2,Basic Network และ Lab Config Cisco


In [6]:
df['score'] = df['score'].astype(str)

df['score_encoded'] = df['score'].str.replace('1','Bad')
df['score_encoded'] = df['score_encoded'].str.replace('2','Bad')
df['score_encoded'] = df['score_encoded'].str.replace('3','Natural')
df['score_encoded'] = df['score_encoded'].str.replace('4','Good')
df['score_encoded'] = df['score_encoded'].str.replace('5','Good')

conditions = [
    (df['score_encoded'] == 'Bad'),
    (df['score_encoded'] == 'Natural'),
    (df['score_encoded'] == 'Good'),
    ]

values = ['0', '1', '2']

df['Label_Enc'] = np.select(conditions, values)

In [7]:
def replace_url(text):
    URL_PATTERN = r"""(?i)\b((?:https?:(?:/{1,3}|[a-z0-9%])|[a-z0-9.\-]+[.](?:com|net|org|edu|gov|mil|aero|asia|biz|cat|coop|info|int|jobs|mobi|museum|name|post|pro|tel|travel|xxx|ac|ad|ae|af|ag|ai|al|am|an|ao|aq|ar|as|at|au|aw|ax|az|ba|bb|bd|be|bf|bg|bh|bi|bj|bm|bn|bo|br|bs|bt|bv|bw|by|bz|ca|cc|cd|cf|cg|ch|ci|ck|cl|cm|cn|co|cr|cs|cu|cv|cx|cy|cz|dd|de|dj|dk|dm|do|dz|ec|ee|eg|eh|er|es|et|eu|fi|fj|fk|fm|fo|fr|ga|gb|gd|ge|gf|gg|gh|gi|gl|gm|gn|gp|gq|gr|gs|gt|gu|gw|gy|hk|hm|hn|hr|ht|hu|id|ie|il|im|in|io|iq|ir|is|it|je|jm|jo|jp|ke|kg|kh|ki|km|kn|kp|kr|kw|ky|kz|la|lb|lc|li|lk|lr|ls|lt|lu|lv|ly|ma|mc|md|me|mg|mh|mk|ml|mm|mn|mo|mp|mq|mr|ms|mt|mu|mv|mw|mx|my|mz|na|nc|ne|nf|ng|ni|nl|no|np|nr|nu|nz|om|pa|pe|pf|pg|ph|pk|pl|pm|pn|pr|ps|pt|pw|py|qa|re|ro|rs|ru|rw|sa|sb|sc|sd|se|sg|sh|si|sj|Ja|sk|sl|sm|sn|so|sr|ss|st|su|sv|sx|sy|sz|tc|td|tf|tg|th|tj|tk|tl|tm|tn|to|tp|tr|tt|tv|tw|tz|ua|ug|uk|us|uy|uz|va|vc|ve|vg|vi|vn|vu|wf|ws|ye|yt|yu|za|zm|zw)/)(?:[^\s()<>{}
]+|[^\s()]*?\)|)+(?:[^\s()]*?\)||[^\s`!()
{};:'".,<>?«»“”‘’])|(?:(?<!@)[a-z0-9]+(?:[.\-][a-z0-9]+)*[.](?:com|net|org|edu|gov|mil|aero|asia|biz|cat|coop|info|int|jobs|mobi|museum|name|post|pro|tel|travel|xxx|ac|ad|ae|af|ag|ai|al|am|an|ao|aq|ar|as|at|au|aw|ax|az|ba|bb|bd|be|bf|bg|bh|bi|bj|bm|bn|bo|br|bs|bt|bv|bw|by|bz|ca|cc|cd|cf|cg|ch|ci|ck|cl|cm|cn|co|cr|cs|cu|cv|cx|cy|cz|dd|de|dj|dk|dm|do|dz|ec|ee|eg|eh|er|es|et|eu|fi|fj|fk|fm|fo|fr|ga|gb|gd|ge|gf|gg|gh|gi|gl|gm|gn|gp|gq|gr|gs|gt|gu|gw|gy|hk|hm|hn|hr|ht|hu|id|ie|il|im|in|io|iq|ir|is|it|je|jm|jo|jp|ke|kg|kh|ki|km|kn|kp|kr|kw|ky|kz|la|lb|lc|li|lk|lr|ls|lt|lu|lv|ly|ma|mc|md|me|mg|mh|mk|ml|mm|mn|mo|mp|mq|mr|ms|mt|mu|mv|mw|mx|my|mz|na|nc|ne|nf|ng|ni|nl|no|np|nr|nu|nz|om|pa|pe|pf|pg|ph|pk|pl|pm|pn|pr|ps|pt|pw|py|qa|re|ro|rs|ru|rw|sa|sb|sc|sd|se|sg|sh|si|sj|Ja|sk|sl|sm|sn|so|sr|ss|st|su|sv|sx|sy|sz|tc|td|tf|tg|th|tj|tk|tl|tm|tn|to|tp|tr|tt|tv|tw|tz|ua|ug|uk|us|uy|uz|va|vc|ve|vg|vi|vn|vu|wf|ws|ye|yt|yu|za|zm|zw)\b/?(?!@)))"""
    return re.sub(URL_PATTERN, 'xxurl', text)

def replace_rep(text):
    def _replace_rep(m):
        c,cc = m.groups()
        return f'{c}xxrep'
    re_rep = re.compile(r'(\S)(\1{2,})')
    return re_rep.sub(_replace_rep, text)

def process_text(text):
    res = text.lower().strip()
    res = replace_url(res)
    res = replace_rep(res)

    res = [word for word in word_tokenize(res) if word and not re.search(pattern=r"\s+", string=word)]
    return res

In [8]:
df.head(1)

,text,score,course_name,score_encoded,Label_Enc
0,โค้ชเอ๋แน่นมากทั้ง MM Mind Set และ Method เทีย...,5,ทำกำไรในตลาดทองอย่างยั่งยืนด้วยเทคนิค 3M,Good,2


In [9]:
def select_label(label):
    df['text'] = df['text'].map(lambda x: '|'.join(process_text(x)))
    ws_tfidf = TfidfVectorizer(tokenizer=word_tokenize, ngram_range=(1, 2), sublinear_tf=True)
    ws_vec = ws_tfidf.fit_transform(df['text']).toarray()
    X = ws_vec
    y = df[str(label)]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15)

    model = LogisticRegression(class_weight='balanced')
    model.fit(X_train, y_train)
    model_pred = model.predict(X_test)
    print("Logistic Regression")
    print(f"{classification_report(y_test, model_pred)}\n")

    clf = RandomForestClassifier(class_weight='balanced')
    clf.fit(X_train, y_train)
    clf_model_pred = clf.predict(X_test)
    print("Random Forest Classifier")
    print(f"{classification_report(y_test, clf_model_pred)}\n")


    SVM_CLF = svm.SVC(class_weight='balanced')
    SVM_CLF.fit(X_train, y_train)
    svm_pred = SVM_CLF.predict(X_test)
    print("Support Vector Machine")
    print(f"{classification_report(y_test, svm_pred)}\n")


In [10]:
df.columns

Index(['text', 'score', 'course_name', 'score_encoded', 'Label_Enc'], dtype='object')

In [11]:
select_label('Label_Enc')

Logistic Regression
              precision    recall  f1-score   support

           0       0.77      0.71      0.74        14
           1       0.43      0.27      0.33        11
           2       0.94      0.98      0.96       125

    accuracy                           0.90       150
   macro avg       0.71      0.65      0.68       150
weighted avg       0.89      0.90      0.89       150


Random Forest Classifier
              precision    recall  f1-score   support

           0       0.62      0.36      0.45        14
           1       0.33      0.09      0.14        11
           2       0.89      0.99      0.94       125

    accuracy                           0.87       150
   macro avg       0.62      0.48      0.51       150
weighted avg       0.83      0.87      0.84       150


Support Vector Machine
              precision    recall  f1-score   support

           0       0.70      0.50      0.58        14
           1       0.67      0.18      0.29        11
     